In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import librosa
import nltk
from nltk.corpus import stopwords

In [26]:
# Load the movie_lines.tsv file into a pandas dataframe
data = pd.read_csv('movie_lines.tsv', sep='\t', encoding='ISO-8859-1', error_bad_lines=False)

C:\Users\siddh\AppData\Local\Temp\ipykernel_18872\2174132205.py:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = pd.read_csv('movie_lines.tsv', sep='\t', encoding='ISO-8859-1', error_bad_lines=False)
Skipping line 32288: expected 5 fields, saw 7
Skipping line 32351: expected 5 fields, saw 6
Skipping line 32390: expected 5 fields, saw 6
Skipping line 32583: expected 5 fields, saw 6
Skipping line 32585: expected 5 fields, saw 6
Skipping line 35684: expected 5 fields, saw 6
Skipping line 62132: expected 5 fields, saw 6
Skipping line 86637: expected 5 fields, saw 6
Skipping line 86722: expected 5 fields, saw 6
Skipping line 86914: expected 5 fields, saw 6
Skipping line 86960: expected 5 fields, saw 6
Skipping line 87010: expected 5 fields, saw 6
Skipping line 87011: expected 5 fields, saw 6
Skipping line 87086: expected 5 fields, saw 6
Skipping line 120607: expected 5 fields, saw 6
Skippin

In [31]:
data.columns = ['lineID', 'characterID', 'movieID', 'character', 'text']

In [32]:
# Remove missing values
data.dropna(inplace=True)

In [33]:
# Create a dictionary to map each label to a number
labels = ['not hate', 'hate']
label_dict = {}
for index, label in enumerate(labels):
    label_dict[label] = index

In [34]:
# Map the labels to numbers
data['label'] = data['text'].apply(lambda x: label_dict['hate'] if 'hate' in x.lower() else label_dict['not hate'])

In [37]:
# Split the dataset into training and testing sets
train_size = int(len(data) * 0.8)
train_df = data[:train_size]
test_df = data[train_size:]

# Set the maximum number of words and sequence length
max_words = 10000
max_len = 200

In [38]:
# Tokenize the text data
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(train_df['text'])
train_sequences = tokenizer.texts_to_sequences(train_df['text'])
test_sequences = tokenizer.texts_to_sequences(test_df['text'])

In [39]:
# Pad the sequences to the same length
train_data = pad_sequences(train_sequences, maxlen=max_len)
test_data = pad_sequences(test_sequences, maxlen=max_len)

In [40]:
# Define the NLP model architecture
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(max_words, 128, input_length=max_len),
    tf.keras.layers.Conv1D(32, 5, activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=4),
    tf.keras.layers.LSTM(128),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_data, train_df['label'], epochs=10, validation_data=(test_data, test_df['label']))

# Test the model
loss, accuracy = model.evaluate(test_data, test_df['label'])
print('Test accuracy:', accuracy)

Epoch 1/10
7216/7216 [==============================] - 479s 66ms/step - loss: 0.0102 - accuracy: 0.9976 - val_loss: 8.1355e-04 - val_accuracy: 0.9999
Epoch 2/10
7216/7216 [==============================] - 481s 67ms/step - loss: 0.0011 - accuracy: 0.9998 - val_loss: 7.0884e-04 - val_accuracy: 0.9999
Epoch 3/10
7216/7216 [==============================] - 493s 68ms/step - loss: 4.0002e-04 - accuracy: 1.0000 - val_loss: 7.1868e-04 - val_accuracy: 0.9999
Epoch 4/10
 618/7216 [=>............................] - ETA: 7:00 - loss: 3.2654e-05 - accuracy: 1.0000